In [85]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [86]:
df = pd.read_csv('churn.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [87]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
y_test = df_test.churn.values
numerical = ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [88]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical+numerical].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)
    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    return dv, model

def predict(df, dv, model):
    dicts = df[categorical+numerical].to_dict(orient='records')
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]
    return y_pred

In [89]:
from tqdm.auto import tqdm

for C in [0.01, 0.001, 0.1, 0.5, 1, 5, 10]:

    scores= []
    kfold = KFold(n_splits=5, shuffle=True, random_state=1)

    for train_idx, val_idx in tqdm(kfold.split(df_full_train)):
        df_train = df_full_train.iloc[train_idx]
        df_val = df_full_train.iloc[val_idx]
        y_train = df_train.churn
        dv, model = train(df_train, y_train, C=C)
        y_pred = predict(df_val, dv, model)
        roc_auc = roc_auc_score(df_val.churn, y_pred)
        scores.append(roc_auc)
    
    print(f'{C=} {np.mean(scores):.3f} +- {np.std(scores):.3f}') 

5it [00:01,  4.05it/s]


C=0.01 0.840 +- 0.009


5it [00:01,  4.00it/s]


C=0.001 0.825 +- 0.009


5it [00:01,  3.84it/s]


...

C=1 0.841 +- 0.008


5it [00:01,  3.91it/s]


C=5 0.841 +- 0.007


5it [00:01,  4.02it/s]

C=10 0.841 +- 0.007


In [90]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
print(auc)

0.8572386167896259


Saving the model

In [91]:
import pickle

In [92]:
output_file = f'model_C={C}.pkl'
output_file

'model_C=10.pkl'

In [93]:
with open(output_file, 'wb') as f_out:
    pickle.dump((dv, model), f_out)

Load the model

In [97]:
import pickle

model_file = f'model_C={C}.pkl'

In [98]:
with open(model_file, 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [103]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=1000))

In [99]:
customer_test = df_test.iloc[0]
#customer_test.to_dict()

customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'no',
    'dependents': 'no',
    'tenure': 41,
    'phoneservice': 'yes',
    'multiplelines': 'no',
    'internetservice': 'dsl',
    'onlinesecurity': 'yes',
    'onlinebackup': 'no',
    'deviceprotection': 'yes',
    'techsupport': 'yes',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contract': 'one_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'bank_transfer_(automatic)',
    'monthlycharges': 79.85,
    'totalcharges': 3320.75,
}

In [107]:
X = dv.transform(customer)
prediction = model.predict_proba(X)[0,1]
prediction

0.06224295541097285